In [22]:
import pandas as pd
# from datetime import datetime
import datetime
import random
import sys


## Job class

Defines 1 job, associated with an aircraft.

The jobs form a linked list, next being a link to the next job in the list for that aircraft.

In [2]:
class job:
    def __init__(self,job_id,description,duration):
        self.id = job_id
        self.description = description
        self.start = 0
        self.duration = duration
        self.aircraft = None
        self.fixed = False
        
#         requiredSkills is a list of required skills
# 1 string per requied. If it is an 'OR' then seprate the skills using '|'
        self.requiredCertifications =[]
    
#     notAllocated is a list of the skills not allocated. Each time a member of staff is allocated, the skill is 
# removed from notAllocated
        self.notAllocated =[]
        self.staff = []
#         A list of staff allocated
    
    def getEnd(self):
        return self.start +self.duration
    
    def __str__(self):
        f = ''
        if self.fixed:
            f='*'
        e= self.getEnd()         
        na =""
        for s in self.notAllocated:
            na = na + s +","
        
        p =""
        for s in self.staff:
            p = p + s.name +","
            
        return f"{self.id} ({self.start}-{e}) {f} NA={na} S= {p}"

    
    def reset(self):
        self.fixed = False
        self.start =0
        self.staff = []
        self.notAllocated = []
        for sk in self.requiredCertifications:
            self.notAllocated.append(sk)
    
    def addCertification(self,skill):
        self.requiredCertifications.append(skill)
        self.reset()
        
    def allocate(self, staffMember):
#         check if skill is needed
        found = False
        i=0
        for sk in self.notAllocated:
            if staffMember.certification in sk:
                found = True
                break
            i=i+1   
        
        if found:
            self.staff.append(staffMember)
            self.fixed = True
            staffMember.jobsAllocated.append(self)
            del self.notAllocated[i]
        
        return found
    
    def check(self, staffMember):
#        Return True if stffMember has the skill needed
        found = False
        i=0
        for sk in self.requiredCertifications:
            if staffMember.certification in sk:
                found = True
                break
            i=i+1   
        return found
    
    def validate(self):
        return None


## Aircraft class

In [3]:
class aircraft:
    def __init__(self,planeID,arrival,departure):
        self.aircraftID = planeID
#         aircraft id
        self.arrival = arrival
#     start of maintaince window
        self.departure = departure
#     end of maintanance window\

        self.jobs =[] #All jobs allocated to this airraft
        self.unscheduled =[] #All jobs not in the job queue
        self.queue =[]
        self.timeAvail = arrival #Time aircraft is available to add the next job
        
        
    def over(self):
        
        if self.available > self.departure:
            return 10
#             d=self.available-self.departure
#             p=d.total_seconds()/60
#             return p*p
        return 0
    
    def validate(self):
        #check times
        buffer = ""
        
        time = self.arrival
        for j in self.queue:
            if j.start < time:
                buffer = buffer + "Time err - " + j.id
            time = j.getEnd()

            
        if len(self.queue)>0:
            if self.available > self.departure:
                buffer = buffer + " Late Departure"
        
        if len(self.unscheduled) >0 :
            buffer = buffer + " Unscheduled jobs."
        if buffer == "":
            return None
            
    
    def add(self,job):
        self.jobs.append(job)
        self.unscheduled.append(job)
        self.reset()
        
        
    def addToQueue(self,job,time=None):
        if job in self.unscheduledJobs:
            self.unscheduledJobs.remove(job)
            
            self.queue.append(job)
            job.fixed = True
            if time != None:
                job.start = time
            else:
                job.start = self.available
            
            self.available = job.getEnd()
                
            

    def reset(self):
        self.available = self.arrival
        self.unscheduledJobs =[]
        self.queue = []
        for j in self.jobs:
            self.unscheduledJobs.append(j)
            
    def __str__(self):
        err =""
        if len(self.queue):
            if self.queue[-1].getEnd() > self.departure:
                err =" LATE DEPARTURE!"

        buffer = self.aircraftID  +" "
        buffer =  buffer + "( "+str(self.arrival)  +":" +str(self.departure) +"Avail:" +str(self.available) + " "+err+" )\n"
        
        buffer = buffer + "Scheduled Jobs \n"
        for j in self.queue:
#         while j != None:
            buffer = buffer +"\t"+ str(j)+"\n"
#             j = j.next
        
        
        if len(self.unscheduledJobs) >0:
            buffer = buffer + "Unscheduled jobs:\n"
            for j in self.unscheduledJobs:
                buffer = buffer + str(j.id) +" : " +j.description +"\n"
                
        return buffer
        


## Staff Class

In [4]:
class staff:
    def __init__(self,name,certification):
        self.name = name
        self.certification = certification
        self.reset()
    
    def allocJob(self, job):
        self.allocJob.append(job)
        
    def reset(self):
        self.jobsAllocated = []
        self.timeAvailable = datetime.datetime(1976, 8, 23)
        
    def validate(self):
        if len(self.jobsAllocated)== 0:
            return None #"OK - No jobs allocated"
        time = self.jobsAllocated[0].start
        buffer = ""
        for j in self.jobsAllocated:
            if j.start < time:
                buffer = buffer + "Start time error " + j.id
            time = j.getEnd()
        
        if buffer == "":
            return None
        else:
            return buffer
        
    def __str__(self):
        buffer = self.name  + " Avail:" + str(self.timeAvailable)
        for j in self.jobsAllocated :
            ac = ""
            if j.aircraft != None:
                ac = j.aircraft.aircraftID
            buffer = buffer +"\n"+ j.id + " "+ac+" "+" ( "+ str(j.start) +" - "+ str(j.getEnd())+" ) "+ j.description 
        return buffer

## Problem Class

In [5]:
class problem:
    def __init__(self):
        self.aircraft ={}
        self.jobs ={}
        self.staff={}
        
    def __str__(self):
        buffer = ""
        for a in self.aircraft:
            buffer = buffer  + str(self.aircraft[a]) +"\n"
        
        return buffer
    
    def getUnallocated(self):
        unalloc = 0
        for j in self.jobs:
            unalloc = unalloc + len(self.jobs[j].notAllocated)
#             p=(len(self.jobs[j].notAllocated)*self.jobs[j].duration.total_seconds() / 60)
#             unalloc = unalloc + (p*p)
        return unalloc
    
    def printStaff(self):
        for s in self.staff:
            print(self.staff[s]+"\n")
            
    def validate(self):
#         Validate whether the solution held is valid
        for a in self.aircraft:
            ac = self.aircraft[a]
            b = ac.validate()
            if b != None:
                print (a +" "+ b)
            
        for j in self.jobs:
            jb = self.jobs[j]
            b = jb.validate()
            if b != None:
                print(j + " "+ b)
            
        for s in self.staff:
            st = self.staff[s]
            b = st.validate()
            if b != None:
                print(s + " "+ b)
            
    def reset(self):
#         Reset any elements of the solution
        for a in self.aircraft:
            ac = self.aircraft[a]
            ac.reset()
            
        for j in self.jobs:
            jb = self.jobs[j]
            jb.reset()
            
        for s in self.staff:
            st = self.staff[s]
            st.reset()
    
    def allocate(self, s, j):
#         print("Alloc debug")
#         Allocate staff to job
#         Check qualifications first

        if j.check(s) == False:
#             print("**1")
            return False
        
        a = j.aircraft
        if j.fixed:
            if s.timeAvailable <= j.start: #a.available:
                j.allocate(s)
                s.timeAvailable = j.getEnd()
#                 a.available = j.getEnd()
#                 print("**2")
                return True
            else:
#                 Staff not available in time
#                 print("**3")
                return False

        if s.timeAvailable < a.available:
#         Staff available before aircraft
            j.allocate(s)
            j.start = a.available
            a.addToQueue(j)
            s.timeAvailable = j.getEnd()
            a.available = j.getEnd()
#             print("**4")
            return True
            
        else:
#             Staff avail after aircraft avail
            j.allocate(s)
            a.addToQueue(j,time=s.timeAvailable)
            s.timeAvailable = j.getEnd()
            a.available = j.getEnd()
#             print("**5")
            return True
            
        
#     def allocate(self, s, j):
# #         Allocate staff to job
#         a = j.aircraft
    
#         if j in a.unscheduled:
#             a.addToQueue(j)
            
#         staffTime = s.timeAvailable
#         jobTime = j.start

#         if jobTime >= staffTime:
#             if j.allocate(s):
#                 s.timeAvailable = j.getEnd()
#                 return True
#             else:
#                 return False
#         else:
#             shiftT = staffTime - jobTime
#             if j.shift(shiftT):
#                 if j.allocate(s):
#                     s.timeAvailable = j.getEnd()
#                     return True
#             return False
        
    def getListJobs(self):
#         Return a complete list of jobs
        return list(self.jobs.keys())
    
    def getListStaff(self):
    #     Retunr a complete list of staff
        return list(self.staff.keys())
        

# Test jobs, staff and Aircraft

## Test Alloc

In [6]:
j= job("J1","Job no 1",10)
j.addCertification("SK1")
j.addCertification("SK2")
print(j)

s1 = staff("Neil","SK1")
if not j.allocate(s1):
    print("Not allocated")
    
s2 = staff("David","SK2")
if not j.allocate(s2):
    print("Not allocated")
    
s3 = staff("Bill","SK2")

if not j.allocate(s3):
    print("Not allocated")
    
print(j)
print(s1)
print(s2)
print(s3)



J1 (0-10)  NA=SK1,SK2, S= 
Not allocated
J1 (0-10) * NA= S= Neil,David,
Neil Avail:1976-08-23 00:00:00
J1   ( 0 - 10 ) Job no 1
David Avail:1976-08-23 00:00:00
J1   ( 0 - 10 ) Job no 1
Bill Avail:1976-08-23 00:00:00


## Test Aircraft and Jobs

In [7]:


ac = aircraft("ac1",0,150)
j1= job("J1","Job no 1",10)
ac.add(j1)

j2= job("J2","Job no 2",20)
ac.add(j2)

j3= job("J3","Job no 3",10)
ac.add(j3)

j4= job("J4","Job no 4",15)
ac.add(j4)

j5= job("J5","Job no 5",10)
ac.add(j5)

j6= job("J6","Job no 6",20)
ac.add(j6)

j7= job("J7","Job no 7",10)
ac.add(j7)

j8= job("J8","Job no 8",15)
ac.add(j8)

ac.addToQueue(j1)
ac.addToQueue(j2)
ac.addToQueue(j3)
ac.addToQueue(j4)
ac.addToQueue(j5)
ac.addToQueue(j6)
ac.addToQueue(j7)
ac.addToQueue(j8)

print(ac)
ac.validate()

ac1 ( 0:150Avail:110  )
Scheduled Jobs 
	J1 (0-10) * NA= S= 
	J2 (10-30) * NA= S= 
	J3 (30-40) * NA= S= 
	J4 (40-55) * NA= S= 
	J5 (55-65) * NA= S= 
	J6 (65-85) * NA= S= 
	J7 (85-95) * NA= S= 
	J8 (95-110) * NA= S= 



## Load Data

In [8]:
file = './Example_Case.xlsx'
planes = pd.read_excel(file, 
                        sheet_name = 0)
planes

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [ ]:
file = './Example_Case.xlsx'
work_packages = pd.read_excel(file, 
                        sheet_name = 1, 
                        index_col = 0)
work_packages

In [ ]:
file = './Example_Case.xlsx'
people = pd.read_excel(file, 
                        sheet_name = 2, 
                        index_col = 0)
people

## Create Problem instance

In [ ]:
instance = problem()
for index, row in people.iterrows():
    instance.staff[row['NAME']] = staff(row['NAME'],row['Certification'])
    

In [ ]:
# Create Plane object and add Jobs

def getDateTime(dateStr,timeStr):
#     Create a dateTime obect based on the date and time strings
    date = datetime.datetime.strptime(dateStr, date_format)
    time = datetime.datetime.strptime(timeStr, time_format)
    time_change = datetime.timedelta(minutes=time.minute,hours=time.hour) 
    date = date + time_change
    return date
    
# print(planes.columns)

date_format = '%Y-%m-%d %H:%M:%S'
time_format =  '%H:%M:%S'

for index, row in planes.iterrows():
    landingTime = getDateTime(str(row['A/C Landing Date']),str(row['A/C Landing Time']))
    departTime = getDateTime(str(row['A/C departure Date']),str(row['A/C departure Time']))
    a= aircraft(str(row['Aicraft (A/C) Serial Number']),landingTime,departTime)
    
#     Add jobs
    jobs = row['Work that needs to be carried out'].split(',')
    for jID in jobs:
        jID= jID.strip()
        r = work_packages.loc[jID]
        duration = datetime.timedelta(minutes=int(r['Minutes']))
        j= job(jID,r['WP description'],duration)
        j.aircraft = a
        certs = r['Required Certified Personnnel'].split(',')
        
#         print(certs)
        for c in certs:
            j.addCertification(c)
        a.add(j)
        instance.jobs[a.aircraftID+"."+j.id] =j
    instance.aircraft[a.aircraftID] = a
    print("Aircraft ")
    print(a)
 

## Allocate staff to jobs

In [10]:
instance.reset()
print("Allocating...")

# instance.allocate(instance.staff['AA1'],instance.jobs['116.WP16'])
# instance.allocate(instance.staff['AA1'],instance.jobs['104.WP18'])


# print("@1\n"+str(instance.staff['NBU9']))



instance.allocate(instance.staff['AA3'],instance.jobs[ '113.WP18'])
print("@1\n"+str(instance.staff['AA16']))

instance.allocate(instance.staff['AA16'],instance.jobs[ '113.WP18'])
print("@2\n"+str(instance.staff['AA16']))

instance.allocate(instance.staff['AA16'],instance.jobs[ '108.WP15'])
print("@3\n"+str(instance.staff['AA16']))


print("\n\n")
instance.validate()


In [12]:
instance.validate()
# print(instance.staff['NBU9'])


In [13]:
print (instance.getListJobs())
print (instance.getListStaff())


## Generate random solution and evaluate

In [14]:

def randSol():
    instance.reset()
    availStaff = instance.getListStaff()
    
    allJobs = instance.getListJobs()
    sol = []
    for j in allJobs:
        theJob = instance.jobs[j]
        c=1
        for c in range(len(theJob.notAllocated)):
            
            rStaff = random.choice(availStaff)
            while not theJob.check(instance.staff[rStaff]):
                rStaff = random.choice(availStaff)
                
            code= j+":"+str(c)
            t = [rStaff,code]
            sol.append(t)
            c=c+1
    random.shuffle(sol)
    return sol

def randSolOLD():
    instance.reset()
    availStaff = instance.getListStaff()
    
    allJobs = instance.getListJobs()
    sol = []
    for j in allJobs:
        theJob = instance.jobs[j]
        c=1
        for c in range(len(theJob.notAllocated)):
            rStaff = random.choice(availStaff)
            code= j+":"+str(c)
            t = [rStaff,code]
            sol.append(t)
            c=c+1
    random.shuffle(sol)
    return sol

sol = randSol()
# print(evaluate(randSol()))
# print(evaluate(randSolOLD()))

NameError: name 'instance' is not defined

In [15]:
def evaluate(sol):
    instance.reset()
    for g in sol:
        j=g[1].split(":")[0]
        instance.allocate(instance.staff[g[0]],instance.jobs[j])

    instance.validate()
    f =instance.getUnallocated()
    for ac in instance.aircraft:
        airc = instance.aircraft[ac]
        f = f +airc.over()

    return f

print(evaluate(sol))





NameError: name 'sol' is not defined

In [16]:
def mutate(genome):
    
    ch = random.randint(0,3)
#     print(ch)
    if ch ==1:
#         print("OK")
        n = random.randint(0,len(genome)-1)
        availStaff = instance.getListStaff()
        
        j =  genome[n][1].split(":")[0]
        theJob = instance.jobs[j]
#         print("****")
#         print(theJob)
        rStaff = random.choice(availStaff)
        while not theJob.check(instance.staff[rStaff]):
                rStaff = random.choice(availStaff)
        genome[n][0]= rStaff
        
#         print("DONE")
#         genome[n][0]= random.choice(availStaff)
    if ch==2:
        x = random.randint(0,len(genome)-1)
        y = random.randint(0,len(genome)-1)
        t = genome[x]
        genome.pop(x)
        genome.insert(y,t)
    if ch == 3:
        genome = timeMutate(genome)
    return genome


def timeMutate(genome):
    tl = datetime.timedelta(minutes=0)
    gene = None
    
    for c  in range(10):        
        x = random.randint(0,len(genome)-1)        
        g = genome[x]
        jb = g[1].split(':')[0]
#         print(instance.jobs[jb].duration)
        if instance.jobs[jb].duration > tl:
            tl = instance.jobs[jb].duration
            gene = x
    
#     print("--")
#     print(tl)
    t = genome[gene]
    genome.pop(gene)
    genome.insert(0,t)
    return genome

sol = randSol()
# print(sol[0])

for c in range(100):
    sol = mutate(sol)
    print(evaluate(sol))
    # print("---")



NameError: name 'instance' is not defined

## Simple Hill Climber

In [17]:
sol = randSol()
print(len(sol))

NameError: name 'instance' is not defined

In [18]:
def copyG(genome):
    n = []
    for g in genome:
        n.append(g.copy())
    
    return n

s2 = copyG(sol)

NameError: name 'sol' is not defined

In [19]:
# sol = randSol()
# print(len(sol))
# #Check len

# best = evaluate(sol)
# print("Start = "+str(best))
# for c in range(100000):
# #     print(c)
#     nSol = copyG(sol)
#     nSol = mutate(nSol)

#     nBest = evaluate(nSol)
#     if nBest < best:
#         print(nBest)
#         sol = nSol.copy()
#         best = nBest
        
# print("Done " +str(best))
# print(len(sol))

## Simple EA

In [20]:
def contains(genome, jCode):
    for j in genome:
        if j[1] == jCode:
            return True
    return False


def xo(pA,pB):
    if len(pA) != len(pB):
        print("Parent len mismatch")
  
    child = []
    for c in  range(len(pA)):
        if not contains(child,pA[c][1]):
                child.append(pA[c].copy())
        if not contains(child,pB[c][1]):
                child.append(pB[c].copy())
    
    return child

s =randSol()

pA= randSol()
pB= randSol()

print(len(pA))
print(len(pB))

child = xo(pA,pB)
print(len(child))
print(evaluate(pA))
print(evaluate(pB))
print(evaluate(child))


NameError: name 'instance' is not defined

In [25]:

    
def tour(pop):
    p1 = random.choice(pop)
    p2 = random.choice(pop)
    
    if p1[0] < p2[0]:
        return p1
    else:
        return p2
        
def rip(pop):
    p1 = random.choice(pop)
    p2 = random.choice(pop)
    
    if p1[0] >  p2[0]:
        return p1
    else:
        return p2
          
    
pop_size = 5000

# bestF = sys.maxsize
# bestI = None
best = None

pop = []
for c in range(0,pop_size):

    i = randSol()
    f= evaluate(i)
    p= (f,i)
    pop.append(p)
    if len(pop)==1:
        best=p
    
    
    if f < best[0]:
        best=p
    
#         bestF = f
#         bestI = i
        
print("Init")
print(best[0])

timeOut = 50000
evals = timeOut
iters=0
while (evals > 0):
    if random.choice([True,False]):
        parent = tour(pop)
        ng = copyG(parent[1])
    else:
        ng = xo(tour(pop)[1],tour(pop)[1])
        
    mutate(ng)
    nf = evaluate(ng)
    evals = evals -1
    child = (nf,ng)

    toGo = rip(pop)
    if toGo[0] > child[0]:
        pop.remove(toGo)
        pop.append(child)
        if child[0] < best[0]:
            best = (child[0],copyG(child[1]))
            evals = timeOut
            print("New best " +str(+best[0]))
              
print("Done :" +str(best[0]))

Init
64
New best 63
New best 60
New best 57
New best 55
New best 40
New best 39
New best 38
New best 37
New best 36
New best 35
New best 34
New best 29
New best 28
New best 27
New best 25
New best 24
New best 23
New best 21
New best 19
New best 18
New best 16
New best 15
New best 14
New best 13
New best 12
New best 11
New best 10
New best 9
New best 8
New best 7
New best 6
New best 5
New best 4
New best 3
New best 2
Done :2


In [26]:
instance.reset()
evaluate(best[1])
print(instance)

101 ( 2025-04-01 06:05:00:2025-04-01 09:55:00Avail:2025-04-01 07:45:00  )
Scheduled Jobs 
	WP1 (2025-04-01 06:05:00-2025-04-01 06:45:00) * NA= S= AA20,AA5,
	WP3 (2025-04-01 06:45:00-2025-04-01 07:45:00) * NA= S= AA17,AA13,

102 ( 2025-04-01 07:10:00:2025-04-01 14:50:00Avail:2025-04-01 09:10:00  )
Scheduled Jobs 
	WP15 (2025-04-01 07:10:00-2025-04-01 08:30:00) * NA= S= AA14,AA8,AA3,
	WP2 (2025-04-01 08:30:00-2025-04-01 09:10:00) * NA= S= AA4,AA12,

103 ( 2025-04-01 08:50:00:2025-04-01 16:30:00Avail:2025-04-01 12:50:00  )
Scheduled Jobs 
	WP17 (2025-04-01 08:50:00-2025-04-01 10:10:00) * NA= S= BA5,AA2,BA6,
	WP7 (2025-04-01 11:05:00-2025-04-01 11:35:00) * NA= S= BA3,BA4,
	WP4 (2025-04-01 12:20:00-2025-04-01 12:50:00) * NA= S= AA6,AA19,

104 ( 2025-04-01 09:15:00:2025-04-01 16:55:00Avail:2025-04-01 12:45:00  )
Scheduled Jobs 
	WP18 (2025-04-01 09:15:00-2025-04-01 10:35:00) * NA= S= AA11,AA7,AA1,
	WP11 (2025-04-01 10:35:00-2025-04-01 11:05:00) * NA= S= BA3,BA9,
	WP10 (2025-04-01 11:05:00-20